# Metaclasses

Metaclasses in Python are a powerful tool that can be used to customize class creation. They allow you to <u>control the behavior of classes and objects in a more flexible way than traditional class inheritance</u>. 

Metaclasses provide a way to customize class creation and behavior in Python, offering a powerful tool for advanced programming tasks.

Here below the examples of metaclasses application:

- [Enforcing Coding Conventions](#enforcing-coding-conventions)
- [Singleton Pattern](#singleton-pattern)
- [Automatic Attribute Initialization](#automatic-attribute-initialization)
- [Method Injection](#method-injection)
- [Tracking Subclasses](#tracking-subclasses)

Other parts of interest:

- [Notes](#notes)
  - [Class Methods](#class-methods)
- [Playground](#playground)

References:

- [Python Metaclasses](https://jfreeman.dev/blog/2020/12/07/python-metaclasses/)
- [Understanding Object Instantiation and Metaclasses in Python](https://www.honeybadger.io/blog/python-instantiation-metaclass/)

## Metaclass

The metaclass arguments for `__new__` method are:

- `name`: the name of the class belonging to the metaclass
- `bases`: list of inherited classes
- `dct`: all the attributes and methods of the class

Here below an example that shows in details the mechanism:

In [15]:
from pprint import pprint

class Metaclass(type):
    def __new__(cls, name, bases, dct):
        print(f"name: {name}")
        print(f"bases: {bases}")
        print("dct:")
        pprint(dct)
        new_dct = {}
        for key, value in dct.items():
            if not key.startswith('__'):
                key = key.upper()
            new_dct[key] = value
        return super().__new__(cls, name, bases, new_dct)
    
class A:
    pass

class MyClass(A, metaclass=Metaclass):
    FOO = 'bar'
    BAR = 'baz'
    foo1 = 'bar'

    def pippo(self):
        pass

    @staticmethod
    def pippo_static():
        pass

name: MyClass
bases: (<class '__main__.A'>,)
dct:
{'BAR': 'baz',
 'FOO': 'bar',
 '__module__': '__main__',
 '__qualname__': 'MyClass',
 'foo1': 'bar',
 'pippo': <function MyClass.pippo at 0x000002587DCBB100>,
 'pippo_static': <staticmethod(<function MyClass.pippo_static at 0x000002587DCBB1A0>)>}


## Enforcing Coding Conventions

A common use of metaclasses is to enforce coding conventions, such as ensuring all class attribute names are uppercase.

In [1]:
class UpperCaseAttributesMeta(type):
    def __new__(cls, name, bases, dct):
        new_dct = {}
        for key, value in dct.items():
            if not key.isupper() and not key.startswith('__'):
                raise ValueError(f"Attribute '{key}' is not uppercase.")
            new_dct[key] = value
        return super().__new__(cls, name, bases, new_dct)

class MyClass(metaclass=UpperCaseAttributesMeta):
    FOO = 'bar'
    BAR = 'baz'
    # foo = 'bar'  # This would raise a ValueError

# This will succeed
print(MyClass.FOO)  # Output: bar

# This will fail at class definition time if uncommented
class AnotherClass(metaclass=UpperCaseAttributesMeta):
    foo = 'bar'  # This will raise a ValueError


bar


ValueError: Attribute 'foo' is not uppercase.

In [13]:
class AllUpperCaseAttributesMeta(type):
    def __new__(cls, name, bases, dct):
        new_dct = {}
        for key, value in dct.items():
            if not key.startswith('__'):
                key = key.upper()
            new_dct[key] = value
        return super().__new__(cls, name, bases, new_dct)

class MyClassUpper(metaclass=AllUpperCaseAttributesMeta):
    FOO = 'bar'
    BAR = 'baz'
    foo1 = 'bar'

print(MyClassUpper.FOO1)
print(MyClassUpper.foo1) # this raises an error since the attribute name has been converted to upper-case

bar


AttributeError: type object 'MyClassUpper' has no attribute 'foo1'

## Singleton Pattern

The Singleton pattern ensures a class has only one instance and provides a global point of access to it. This can be implemented using a metaclass.

In [ ]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class SingletonClass(metaclass=SingletonMeta):
    def __init__(self, value):
        self.value = value  

# Both instances are actually the same
instance1 = SingletonClass(10)
instance2 = SingletonClass(20)

print(instance1.value)  # Output: 10
print(instance2.value)  # Output: 10
print(instance1 is instance2)  # Output: True


## Automatic Attribute Initialization

You can use a metaclass to automatically initialize attributes based on keyword arguments.

In [ ]:
class AutoInitMeta(type):
    def __call__(cls, *args, **kwargs):
        instance = super().__call__(*args, **kwargs)
        for key, value in kwargs.items():
            setattr(instance, key, value)
        return instance

class AutoInitClass(metaclass=AutoInitMeta):
    def __init__(self, *args, **kwargs):
        self.initialized = True

# Automatically initialize attributes
obj = AutoInitClass(foo='bar', spam='eggs')

print(obj.foo)          # Output: bar
print(obj.spam)         # Output: eggs
print(obj.initialized)  # Output: True

## Method Injection

You can use a metaclass to inject methods into a class.

In [ ]:
def injected_method(self):
    return f"This method was injected into {self.__class__.__name__}"

class MethodInjectionMeta(type):
    def __new__(cls, name, bases, dct):
        dct['injected_method'] = injected_method
        return super().__new__(cls, name, bases, dct)

class MyClassWithInjection(metaclass=MethodInjectionMeta):
    pass

obj = MyClassWithInjection()
print(obj.injected_method())  # Output: This method was injected into MyClassWithInjection

## Tracking Subclasses

You can use a metaclass to automatically keep track of subclasses.

In [ ]:
class SubclassTrackerMeta(type):
    def __init__(cls, name, bases, dct):
        if not hasattr(cls, 'subclasses'):
            cls.subclasses = []
        else:
            cls.subclasses.append(cls)
        super().__init__(name, bases, dct)

class Base(metaclass=SubclassTrackerMeta):
    pass

class SubClass1(Base):
    pass

class SubClass2(Base):
    pass

print(Base.subclasses)  # Output: [<class '__main__.SubClass1'>, <class '__main__.SubClass2'>]


# Notes

There are at least two ways you can change the process of class object creation:

1. by using class decorators
2. by explicitly specifying a metalcass

A `metaclass` looks like a regular class, and the only exception is that it has to inherit a `type` class, because `type` classes have all the implementation that is required for our code to still work as expected. 

Here below `MyMeta` is the driving force behind new class object instantiation and also specifies how new class instances are created.

In [1]:
class MyMeta(type):
    def __call__(self, *args, **kwargs):
        print(f'{self.__name__} is called'
              f' with args={args}, kwargs={kwargs}')

class Parent(metaclass=MyMeta):
    def __new__(cls, name, age):
        print('new is called')
        return super().__new__(cls)

    def __init__(self, name, age):
        print('init is called')
        self.name = name
        self.age = age

parent = Parent('John', 35)
# Parent is called with args=('John', 35), kwargs={}
type(parent)
# NoneType

Parent is called with args=('John', 35), kwargs={}


NoneType

`parent` holds nothing, because `MyMeta.__call__` just prints information and returns nothing. Explicitly, that is. Implicitly, that means that it returns `None`, which is of `NoneType`.

Here below it is fixed:

In [2]:
class MyMeta(type):

    def __call__(cls, *args, **kwargs):
        print(f'{cls.__name__} is called'
              f'with args={args}, kwargs={kwargs}')
        print('metaclass calls __new__')
        obj = cls.__new__(cls, *args, **kwargs)

        if isinstance(obj, cls):
            print('metaclass calls __init__')
            cls.__init__(obj, *args, **kwargs)

        return obj

class Parent(metaclass=MyMeta):

    def __new__(cls, name, age):
        print('new is called')
        return super().__new__(cls)

    def __init__(self, name, age):
        print('init is called')
        self.name = name
        self.age = age

parent = Parent('John', 35)
# Parent is called with args=('John', 35), kwargs={}
# metaclass calls __new__
# new is called
# metaclass calls __init__
# init is called

type(parent)
# Parent

str(parent)
# '<__main__.Parent object at 0x103d540a0>'

Parent is calledwith args=('John', 35), kwargs={}
metaclass calls __new__
new is called
metaclass calls __init__
init is called


'<__main__.Parent object at 0x000002265CCA9DC0>'

## Class Methods

They can be displayed by invoking the `dir` built-in function.

In [3]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [4]:
dir(type)

['__abstractmethods__',
 '__annotations__',
 '__base__',
 '__bases__',
 '__basicsize__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dictoffset__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__flags__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__instancecheck__',
 '__itemsize__',
 '__le__',
 '__lt__',
 '__module__',
 '__mro__',
 '__name__',
 '__ne__',
 '__new__',
 '__or__',
 '__prepare__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasscheck__',
 '__subclasses__',
 '__subclasshook__',
 '__text_signature__',
 '__type_params__',
 '__weakrefoffset__',
 'mro']

# Playground

- Methods to be checked:
  - `__init__`
  - `__call__`
  - `__new__`
- who calls who?

In [ ]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass